In [2]:

import numpy as np
from stdb import load_db 
import pygmt
import pandas as pd
import pickle
import glob, os
import warnings  
from obspy.core.event.catalog import read_events
warnings.filterwarnings("ignore")  


In [34]:
PKLpath = '/Volumes/home/Research/STEP/01_Split'
Path = '/Volumes/home/Research/STEP/02_Station_result_csv/2010-2015_Mw7_100-140degree_SKS'

phase = 'SKS'
Mag = 7
DATApath = f'/Volumes/home/Research/STEP/01_Split/DATA_{phase}_Mw*'


In [35]:
def Calc_rho(RCdt, SCdt):
    rho = RCdt/SCdt 
    return rho
def Calc_Phi(RCPhi, SCPhi):
    Phi = max (abs(RCPhi-SCPhi), abs(SCPhi-RCPhi))
    if Phi > 90: Phi = 180 - Phi
    return Phi
def RCcoeff(DATApath,NET,STA,event):
    RCfile = glob.glob(f'{DATApath}/{NET}.{STA}/{event}/RC_res_manual.pkl')[0]
    RCpkl  = pickle.load(open(RCfile, "rb"))
    coeff  = abs(RCpkl.errc)
    return coeff


In [38]:
for CSVpath in sorted(glob.glob(f'{Path}/I*.csv')):
    filename = CSVpath.rsplit('/')[-1]
    netsta = filename.rsplit('_')[0]
    NET = netsta.rsplit('.')[0] ; STA = netsta.rsplit('.')[1]
    STApkl = pickle.load(open(f'{PKLpath}/{NET}_{STA}.pkl', "rb"))
    values = [value for keys, value in STApkl.items() ][0]
    
    st_lat = values['latitude']; st_lon = values['longitude']
    df = pd.read_csv(CSVpath)
    
    df = df[df['Phase']==phase][df['SNRQ']>=5][df['Pick']==True]
    
    
    for i in range(len(df)):
        evtTime = df["Event"].values[i]
        RCPhi = df['RCPhi'].values[i]
        RCdt  = df['RCdt'].values[i]
        SCPhi = df['SCPhi'].values[i]
        SCdt  = df['SCdt'].values[i]
        
        rho = Calc_rho(RCdt, SCdt)
        phi = Calc_Phi(RCPhi, SCPhi)
        if 25 < phi < 68 or df["CpH"].values[i] >= 0.77: pass
        else: 
            if 0.8 < rho < 1.1 and phi < 8:   ## non-null good condition 
                print(f'{STA} {evtTime} Good')
            elif 0.7 <= rho < 1.2 and phi <=25: ## non-null FAIR condition 
                print(f'{STA} {evtTime} Fair')
                

GNI 20110824_174611 Fair


In [41]:
for CSVpath in sorted(glob.glob(f'{Path}/I*.csv')):
    filename = CSVpath.rsplit('/')[-1]
    netsta = filename.rsplit('_')[0]
    NET = netsta.rsplit('.')[0] ; STA = netsta.rsplit('.')[1]
    STApkl = pickle.load(open(f'{PKLpath}/{NET}_{STA}.pkl', "rb"))
    values = [value for keys, value in STApkl.items() ][0]
    
    st_lat = values['latitude']; st_lon = values['longitude']
    df = pd.read_csv(CSVpath)
    
    df = df[df['Phase']==phase][df['SNRQ']>=5][df['Pick']==True]
    
    
    for i in range(len(df)):
        evtTime = df["Event"].values[i]
        RCPhi = df['RCPhi'].values[i]
        RCdt  = df['RCdt'].values[i]
        RCPhi_std = df['RCPhi_std'].values[i]
        RCdt_std  = df['RCdt_std'].values[i]
        SCPhi = df['SCPhi'].values[i]
        SCdt  = df['SCdt'].values[i]
        SCPhi_std = df['SCPhi_std'].values[i]
        SCdt_std  = df['SCdt_std'].values[i]   
        CpH = df['CpH'].values[i]   

        rho = Calc_rho(RCdt, SCdt)
        phi = Calc_Phi(RCPhi, SCPhi)
        coeff = RCcoeff(DATApath,NET,STA,evtTime)

        if CpH <=0.76 :
            if (SCPhi_std <20 and SCdt_std<0.5 and coeff >=0.8):
                print(f'{STA} {evtTime} Good')
            elif phi <10:
                print(f'{STA} {evtTime} Good')
            elif (SCPhi_std <=30 and SCdt_std<=1.0 and coeff >=0.8) :
                print(f'{STA} {evtTime} Fair')
            elif (10 <= phi <25): 
                print(f'{STA} {evtTime} Fair')
            else:
                # pass
                print(f'{STA} {evtTime} Poor')
        else:
            if CpH >=0.9:
                pass
                

GNI 20110101_095658 Poor
GNI 20110824_174611 Good
